In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,dtype=int, drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
dataset=dataset.drop("User ID",axis=1)

In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [12]:
from sklearn.naive_bayes import GaussianNB

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}



grid = GridSearchCV(GaussianNB(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1')
                   
   
# fitting the model for grid search 
grid.fit(X_train_, y_train) 
 

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             scoring='f1', verbose=3)

In [14]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test_) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'var_smoothing': 1e-09}: 0.9095717304891537


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[81  4]
 [ 8 41]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.91      0.95      0.93        85
           1       0.91      0.84      0.87        49

    accuracy                           0.91       134
   macro avg       0.91      0.89      0.90       134
weighted avg       0.91      0.91      0.91       134



In [18]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.5

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003547,0.001590,0.003705,0.000853,1.000000e-09,{'var_smoothing': 1e-09},0.709677,0.810811,0.702703,0.947368,0.972973,0.828706,0.114244,1
1,0.002786,0.001198,0.004226,0.001065,1.000000e-08,{'var_smoothing': 1e-08},0.709677,0.810811,0.702703,0.947368,0.972973,0.828706,0.114244,1
2,0.001348,0.001130,0.004292,0.001949,1.000000e-07,{'var_smoothing': 1e-07},0.709677,0.810811,0.702703,0.947368,0.972973,0.828706,0.114244,1
3,0.002629,0.001321,0.003055,0.000900,1.000000e-06,{'var_smoothing': 1e-06},0.709677,0.810811,0.702703,0.947368,0.972973,0.828706,0.114244,1
4,0.002417,0.000804,0.003885,0.000979,1.000000e-05,{'var_smoothing': 1e-05},0.709677,0.810811,0.702703,0.947368,0.972973,0.828706,0.114244,1


In [21]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Gender Male 0 or 1:"))

Age: 28
Salary: 20000
Gender Male 0 or 1: 1


In [22]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
